# Despliegue Clasificador XGBoost: Inference Recommender

In [1]:
# librerias

# generales
import pandas as pd
import numpy as np
import joblib


# visualización
import matplotlib.pyplot as plt
from warnings import warn
from pandas.plotting import scatter_matrix


# AWS
from sagemaker import get_execution_role, Session, image_uris
import boto3
import time

# modelos
from lightgbm import LGBMClassifier, LGBMModel
from xgboost import XGBClassifier
import xgboost



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(


In [2]:
# parametros AWS
region = boto3.Session().region_name
role = get_execution_role()
sagemaker_client = boto3.client("sagemaker", region_name=region)
sagemaker_session = Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
# ver lista de modelos Inference Recommender
list_model_metadata_response=sagemaker_client.list_model_metadata()
list_model_metadata_response

{'ModelMetadataSummaries': [{'Domain': 'COMPUTER_VISION',
   'Framework': 'TENSORFLOW',
   'Task': 'IMAGE_CLASSIFICATION',
   'Model': 'efficientnetb7',
   'FrameworkVersion': '1.15.5'},
  {'Domain': 'COMPUTER_VISION',
   'Framework': 'PYTORCH',
   'Task': 'IMAGE_SEGMENTATION',
   'Model': 'unet',
   'FrameworkVersion': '1.6.0'},
  {'Domain': 'MACHINE_LEARNING',
   'Framework': 'XGBOOST',
   'Task': 'CLASSIFICATION',
   'Model': 'xgboost',
   'FrameworkVersion': '1.0-1'},
  {'Domain': 'COMPUTER_VISION',
   'Framework': 'TENSORFLOW',
   'Task': 'OBJECT_DETECTION',
   'Model': 'faster-rcnn-resnet101',
   'FrameworkVersion': '1.15.5'},
  {'Domain': 'COMPUTER_VISION',
   'Framework': 'TENSORFLOW',
   'Task': 'IMAGE_CLASSIFICATION',
   'Model': 'nasnetlarge',
   'FrameworkVersion': '1.15.5'},
  {'Domain': 'COMPUTER_VISION',
   'Framework': 'TENSORFLOW',
   'Task': 'IMAGE_CLASSIFICATION',
   'Model': 'vgg16',
   'FrameworkVersion': '1.15.5'},
  {'Domain': 'COMPUTER_VISION',
   'Framework': '

In [4]:
# parametros de  ML framework 
model_name = "xgboost"
framework = 'XGBOOST'
framework_version = '1.0-1'

# parametros de modelo ML 
ml_domain = 'MACHINE_LEARNING'
ml_task = 'CLASSIFICATION'


In [5]:
# acceder a S3
s3 = boto3.client('s3')
bucket_name = 'finvivirbucket01' # nombre de bucket 

response = s3.list_objects_v2(Bucket=bucket_name)
# ver object key de archivos en bucket
for obj in response['Contents']:
    object_key = obj['Key']
    print(object_key)  # Print the object key of each file
    
# extraer keys de archivos del bucket
data_key = 'AbandonoTC.csv'  # Adjust for different file extensions
x_train_key = 'x_trainpreprocess.csv'
x_test_key = 'x_testpreprocess.csv'
y_train_key = 'y_trainpreprocess.csv'
y_test_key = 'y_testpreprocess.csv'

AbandonoTC.csv
AbandonoTC.xlsx
artifacts/
x_testpreprocess.csv
x_trainpreprocess.csv
y_testpreprocess.csv
y_trainpreprocess.csv


In [6]:
# cargar bases de datos de S3
# Retrieve objects and create DataFrames
data = pd.read_csv(s3.get_object(Bucket=bucket_name, Key=data_key)['Body'])
x_train = pd.read_csv(s3.get_object(Bucket=bucket_name, Key=x_train_key)['Body'])
x_test = pd.read_csv(s3.get_object(Bucket=bucket_name, Key=x_test_key)['Body'])
y_train = pd.read_csv(s3.get_object(Bucket=bucket_name, Key=y_train_key)['Body'])
y_test = pd.read_csv(s3.get_object(Bucket=bucket_name, Key=y_test_key)['Body'])


In [7]:
# cargar modelos pre-entrenados
# mejor modelo escogido para la evaluación final
#modelo_top = joblib.load('modelos/modelo_top.pkl')
grid_xgb = joblib.load('modelos/XGBoostClassifier.pkl')
xgb_op = grid_xgb.best_estimator_
xgb_op

[03:33:24] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1700181168148/work/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



/opt/conda/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator GridSearchCV from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight='balanced', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, device='gpu',
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.505, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=0.01, missing=nan, monotone_constraints=None,
              n_estimators=225, n_jobs=-1, num_parallel_tree=None, ...)

In [8]:
# guardar modelo
model_fname = "xgboost.model"
xgb_op.save_model(model_fname)

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:761: UserWarning: n_estimators is not saved in Scikit-Learn meta.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:761: UserWarning: max_depth is not saved in Scikit-Learn meta.
  warnings.warn(


In [9]:
# crear tarball
model_archive_name = "xgbmodel.tar.gz"
!tar -cvpzf {model_archive_name} 'xgboost.model'
model_archive_name

xgboost.model


'xgbmodel.tar.gz'

In [10]:
# cargar modelo a S3
model_url = sagemaker_session.upload_data(path=model_archive_name, key_prefix="xgbmodel")
print("model uploaded to: {}".format(model_url))

model uploaded to: s3://sagemaker-us-east-1-735423154527/xgbmodel/xgbmodel.tar.gz


In [11]:
# crear archivo de payload
payload_archive_name = "xgb_payload.tar.gz"
print(x_test.shape)
batch_size = 100
batch_size


(2026, 33)


100

In [12]:
np.savetxt("payload_sample.csv", x_test[0:batch_size], delimiter=",")
!wc -l payload_sample.csv

100 payload_sample.csv


In [13]:
# crear tarball
!tar -cvzf {payload_archive_name} payload_sample.csv

payload_sample.csv


In [14]:
# cargar payload a bucket S3
sample_payload_url = sagemaker_session.upload_data(
    path=payload_archive_name, key_prefix="xgb_payload"
)

In [15]:
# cargar imagen del container URL
dlc_uri = image_uris.retrieve("xgboost", region, "1.2-1")
dlc_uri


'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-1'

In [16]:
# crear Model Group
model_package_group_name = "{}-cpu-models-".format(framework) + str(round(time.time()))
model_package_group_description = "{} models".format(ml_task.lower())

model_package_group_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageGroupDescription": model_package_group_description,
}

create_model_package_group_response = sagemaker_client.create_model_package_group(
    **model_package_group_input_dict
)
print(
    "ModelPackageGroup Arn : {}".format(create_model_package_group_response["ModelPackageGroupArn"])
)

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:735423154527:model-package-group/XGBOOST-cpu-models-1705894406


In [17]:
# registrar Model Version/Package
model_package_description = "{} {} inference recommender".format(framework, model_name)

model_approval_status = "PendingManualApproval"

create_model_package_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "Domain": ml_domain.upper(),
    "Task": ml_task.upper(),
    "SamplePayloadUrl": sample_payload_url,
    "ModelPackageDescription": model_package_description,
    "ModelApprovalStatus": model_approval_status,
}
create_model_package_input_dict

{'ModelPackageGroupName': 'XGBOOST-cpu-models-1705894406',
 'Domain': 'MACHINE_LEARNING',
 'Task': 'CLASSIFICATION',
 'SamplePayloadUrl': 's3://sagemaker-us-east-1-735423154527/xgb_payload/xgb_payload.tar.gz',
 'ModelPackageDescription': 'XGBOOST xgboost inference recommender',
 'ModelApprovalStatus': 'PendingManualApproval'}

In [18]:
# configurar parametros de inferencia
input_mime_types = ["text/csv"]
input_mime_types

['text/csv']

In [19]:
supported_realtime_inference_types = [
    "ml.m4.2xlarge",
    "ml.c5.2xlarge",
    "ml.c5.xlarge",
    "ml.c5.9xlarge",
]

In [20]:
modelpackage_inference_specification = {
    "InferenceSpecification": {
        "Containers": [
            {
                "Image": dlc_uri,
                "Framework": framework.upper(),
                "FrameworkVersion": framework_version,
                "NearestModelName": model_name,
            }
        ],
        "SupportedContentTypes": input_mime_types,  # required, must be non-null
        "SupportedResponseMIMETypes": [],
        "SupportedRealtimeInferenceInstanceTypes": supported_realtime_inference_types,  # optional
    }
}

# Specify the model data
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0][
    "ModelDataUrl"
] = model_url


In [21]:
# crear model package/version en SageMaker Model Registry
create_model_package_input_dict.update(modelpackage_inference_specification)
create_mode_package_response = sagemaker_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print("ModelPackage Version ARN : {}".format(model_package_arn))

ModelPackage Version ARN : arn:aws:sagemaker:us-east-1:735423154527:model-package/XGBOOST-cpu-models-1705894406/1


In [22]:
# crear  Inference Recommender Default Job
job_name = model_name + "-instance-" + str(round(time.time()))
job_description = "{} {}".format(framework, model_name)
job_type = "Default"
print(job_name)

# recomendaciones
recom = sagemaker_client.create_inference_recommendations_job(
    JobName=job_name,
    JobDescription=job_description,  # optional
    JobType=job_type,
    RoleArn=role,
    InputConfig={"ModelPackageVersionArn": model_package_arn},
)

print(recom)

xgboost-instance-1705894407
{'JobArn': 'arn:aws:sagemaker:us-east-1:735423154527:inference-recommendations-job/xgboost-instance-1705894407', 'ResponseMetadata': {'RequestId': '07ac8186-0963-47a0-8ded-f1e75879a4a7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '07ac8186-0963-47a0-8ded-f1e75879a4a7', 'content-type': 'application/x-amz-json-1.1', 'content-length': '111', 'date': 'Mon, 22 Jan 2024 03:33:26 GMT'}, 'RetryAttempts': 0}}


In [23]:
"""
finished = False
while not finished:
    inference_recommender_job = sagemaker_client.describe_inference_recommendations_job(JobName=job_name)
    if inference_recommender_job["Status"] in ["COMPLETED", "STOPPED", "FAILED"]:
        finished = True
    else:
        print("In progress")
        time.sleep(300)

if inference_recommender_job["Status"] == "FAILED":
    print("Inference recommender job failed ")
    print("Failed Reason: {}".inference_recommender_job["FailedReason"])
else:
    print("Inference recommender job completed")

data = [
    {**x["EndpointConfiguration"], **x["ModelConfiguration"], **x["Metrics"]}
    for x in inference_recommender_job["InferenceRecommendations"]
]
df = pd.DataFrame(data)
dropFilter = df.filter(["VariantName"])
df.drop(dropFilter, inplace=True, axis=1)
pd.set_option("max_colwidth", 400)
df.head()
    
""" 

'\nfinished = False\nwhile not finished:\n    inference_recommender_job = sagemaker_client.describe_inference_recommendations_job(JobName=job_name)\n    if inference_recommender_job["Status"] in ["COMPLETED", "STOPPED", "FAILED"]:\n        finished = True\n    else:\n        print("In progress")\n        time.sleep(300)\n\nif inference_recommender_job["Status"] == "FAILED":\n    print("Inference recommender job failed ")\n    print("Failed Reason: {}".inference_recommender_job["FailedReason"])\nelse:\n    print("Inference recommender job completed")\n\ndata = [\n    {**x["EndpointConfiguration"], **x["ModelConfiguration"], **x["Metrics"]}\n    for x in inference_recommender_job["InferenceRecommendations"]\n]\ndf = pd.DataFrame(data)\ndropFilter = df.filter(["VariantName"])\ndf.drop(dropFilter, inplace=True, axis=1)\npd.set_option("max_colwidth", 400)\ndf.head()\n    \n'

In [24]:
# ver sub-tareas para Inference Recommender job
list_job_steps_response = sagemaker_client.list_inference_recommendations_job_steps(JobName=job_name)
print(list_job_steps_response)

{'Steps': [], 'ResponseMetadata': {'RequestId': '2cb6f1b3-7874-401f-aa5c-ef49e816d5ca', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2cb6f1b3-7874-401f-aa5c-ef49e816d5ca', 'content-type': 'application/x-amz-json-1.1', 'content-length': '12', 'date': 'Mon, 22 Jan 2024 03:33:28 GMT'}, 'RetryAttempts': 0}}
